# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from seaborn import heatmap
import cv2
import glob as gb 
import random
import os



from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import  LabelEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array, array_to_img, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import callbacks
from tensorflow.math import confusion_matrix
from tensorflow.keras.utils import to_categorical

# Data paths

In [ ]:
train = '/kaggle/input/skin-cancer-malignant-vs-benign/train/'
test = '/kaggle/input/skin-cancer-malignant-vs-benign/test/'

# OpenFolders 

In [ ]:
total = 0 
classes = 0
for  folder in os.listdir(train):
    files =  gb.glob(pathname=str(train +folder+'//*.jpg'))
    print(f'For training data , found {len(files)} in folder {folder}')
    total  = len(files)+total
    classes=1+classes
print(f'we have {(total)} items in train data and we have {classes} classes') 

In [ ]:
totaltest = 0 
classestest = 0
for  folder in os.listdir(test):
    files =  gb.glob(pathname=str(test +folder+'//*.jpg'))
    print(f'For training data , found {len(files)} in folder {folder}')
    totaltest  = len(files)+totaltest
    classestest=1+classestest
print(f'we have {(totaltest)} items in train data and we have {classestest} classes') 

Noticed that the data is samll 

# Resize the data

In [ ]:
image_width = 100
image_height = 100
image_channel = 3 #depth

In [ ]:

x_train=[]
y_train = []
for  folder in os.listdir(train):
    files =  gb.glob(pathname=str(train+folder+'//*.jpg'))
    for file in files:
        img = cv2.imread(file)
        imagearray=cv2.resize(img,(image_width,image_height))
        x_train.append(list(imagearray))
        y_train.append(folder)

        


In [ ]:
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(x_train),9))) : 
    plt.subplot(3,3,n+1)
    plt.imshow(x_train[i])   
    plt.axis('off')
    plt.title(y_train[i])

In [ ]:

x_test=[]
y_test = []
for  folder in os.listdir(test):
    files =  gb.glob(pathname=str(test+folder+'//*.jpg'))
    for file in files:
        img = cv2.imread(file)
        imagearray=cv2.resize(img,(image_width,image_height))
        x_test.append(list(imagearray))
        y_test.append(folder)

In [ ]:
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(x_test),9))) : 
    plt.subplot(3,3,n+1)
    plt.imshow(x_test[i])   
    plt.axis('off')
    plt.title(y_test[i])

# Normalization

In [ ]:
  
x_train = np.array(x_train, dtype=np.float16) / 225.0
x_test = np.array(x_test, dtype=np.float16) / 225.0

# One Hot Encoding

In [ ]:
# One hot encoding on target variable
LE = LabelEncoder()
y_train = to_categorical(LE.fit_transform(y_train))
y_test = to_categorical(LE.fit_transform(y_test))

# Data agumentaion 
to increase the amount of data

In [ ]:
Data_Augmentation = keras.Sequential([
                                      layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical", input_shape=(image_height, image_width, image_channel)),
                                      layers.experimental.preprocessing.RandomRotation(0.2),
                                      layers.experimental.preprocessing.RandomZoom(0.1),
])

In [ ]:


model = keras.models.Sequential([
    Data_Augmentation,
        keras.layers.Conv2D(256,kernel_size=(3,3),activation='relu'),
        keras.layers.MaxPool2D(2,2),
        keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu'),
        keras.layers.MaxPool2D(2,2),
        keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
        keras.layers.Flatten() ,    
        keras.layers.Dense(128,activation='relu') ,    
        keras.layers.Dense(64,activation='relu') ,    
        keras.layers.Dense(32,activation='relu')   ,            
        keras.layers.Dense(2,activation='sigmoid') ,    
        ])


     

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer ='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs =100
batch=128
CNN = model.fit(x_train, y_train,batch_size=batch, epochs=epochs,verbose=1,validation_data=(x_test,y_test))

In [ ]:
loss , acc =model.evaluate(x_test,y_test)

In [ ]:
#Plot the training history
plt.figure(figsize=(12, 5))
plt.plot(CNN.history['accuracy'], color='r')
plt.plot(CNN.history['val_accuracy'], color='b')
plt.title('CNN Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
#Plot the loss history
plt.figure(figsize=(12, 5))
plt.plot(CNN.history['loss'], color='r')
plt.plot(CNN.history['val_loss'], color='b')
plt.title('CNN Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
y_pre = model.predict(x_test)

In [ ]:
y_pre

# Confusion Matrix

In [ ]:
def conf_matrix(y_test, y_pred):
    y_predicted_labels = [np.argmax(i) for i in y_pred]
    y_predicted_labels = np.array(y_predicted_labels)
    y_test = np.argmax(y_test, axis=1)
    cm = confusion_matrix(labels=y_test, predictions=y_predicted_labels)
    plt.figure(figsize=(12,10))
    heatmap(cm, annot=True, fmt='d');

In [ ]:
conf_matrix(y_test, y_pre)

# Classification Report

In [ ]:
def class_report(y_test, y_pred):
    y_predicted_labels = [np.argmax(i) for i in y_pred]
    y_predicted_labels = np.array(y_predicted_labels)
    y_test = np.argmax(y_test, axis=1)
    print(classification_report(y_test, y_predicted_labels))

In [ ]:
class_report(y_test, y_pre)